In [4]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte

In [5]:
read_path = r"C:\ThesisMedia\videos_gopro\GoPro\Cropped images\Extracted frame\2.7k"
write_path = r"C:\ThesisMedia\videos_gopro\GoPro\Cropped images\Cropped_Pralines\2.7K"
box_image_path = r"C:\ThesisMedia\videos_gopro\GoPro\Cropped images\Box_drawings\2.7K"

In [ ]:
### Apply k-means after blur
def kmeans_for_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    vector_img = img.reshape((-1, 3))
    vector_img = np.float32(vector_img)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 5, 1.0)
    K = 3
    attempts= 15
    ret, label, center=cv2.kmeans(vector_img,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    mean_image = res.reshape((img_copy.shape))
    mean_image = cv2.cvtColor(mean_image, cv2.COLOR_BGR2RGB)
    # mean_image = cv2.resize(mean_image, (0,0), fx=0.5, fy=0.5)
    # cv2.imshow('k-mean_image', mean_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    cv2.imwrite('kmeanimg.jpg',mean_image)
    
    return mean_image

In [ ]:
### Read the image from a file
directory = os.fsencode(read_path)
mould_count = 1
crop_count = 1


for file in os.listdir(directory):
    filename = os.fsdecode(file)
    img = cv2.imread(os.path.join(read_path , filename))
    print("processing Mould No: ", mould_count)
    
    img = cv2.GaussianBlur(img,(9,9),0)
    # img = cv2.resize(img, (0,0), fx=0.5, fy=0.5)
    img_copy = img.copy()
    imgcopytocut = img.copy()
    #     cv2.imshow('image', img)
    #     cv2.waitKey()
    #     cv2.destroyAllWindows()

    ### Apply mask to the image
    lower = np.array([0,0,0])
    higher = np.array([120, 140, 150])    #80,255,255
    mask = cv2.inRange(kmeans_for_image(img),lower,higher)
    mask.shape
    # cv2.imshow('image', mask)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    detected_circles = cv2.HoughCircles(mask,
                                        cv2.HOUGH_GRADIENT,
                                        dp=0.78,
                                        minDist=130,
                                        param1=390,
                                        param2=5,
                                        minRadius=56,
                                        maxRadius=63)
    
    if detected_circles is not None:
            crop_count = 1
            num_circles = len(detected_circles)
            # Convert the circle parameters a, b and r to integers.
            detected_circles = np.uint16(np.around(detected_circles))
            # print(detected_circles)
            
            for value in detected_circles[0, :]:
                a, b, r = value[0], value[1], value[2]

                #4k-(800<a<3150 and 280<b<1500)


                if (480<a<2400 and 100<b<1150):                      # Change this when the image changes
                    cv2.circle(img_copy, (a, b), r, (0, 0, 255), 2)
                    cv2.circle(img_copy, (a, b), 1, (0, 0, 255), 3)    # Draw a small circle (of radius 1) to show the center.
                    buffer = int(1.15*r)
                    cv2.rectangle(img_copy,(a-buffer,b-buffer),(a+buffer,b+buffer),(0,255,0),3)
                    cropped_praline = imgcopytocut[(b-buffer):(b+buffer) , (a-buffer):(a+buffer)]
                    # print(cropped_praline.shape)                
                    cv2.imwrite(write_path + '/Mould-%#04d' %mould_count +'-IMG%#05d.jpg' %crop_count ,cropped_praline)
                    crop_count += 1
    # img_copy = cv2.resize(img_copy, (0,0), fx=0.5, fy=0.5)
    else:
        num_circles = 0
    cv2.imwrite(box_image_path + '/Mould-%#04d-%d.jpg' % (mould_count, crop_count-1), img_copy)
    mould_count += 1
    # cv2.imshow("Detected Circle", img_copy)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

print("Completed cropping")